In [ ]:
from IPython.core.display import HTML
with open('../../style.css') as file:
    css = file.read()
HTML(css)

# Embedded Actions in <span style="font-variant:small-caps;">Antlr</span> Grammars

The *pure grammar* is stored in the file `Grammar.g4`. 

In [ ]:
!cat -n Program.g4

The grammar shown above has no *semantic actions* (with the exception of the `skip` action). 
We extend this grammar now with *semantic actions* so that we can actually compute something.
This grammar is stored in the file `Calculator.g4`.  It describes a language for a 
*symbolic calculator*: This calculator is able to evaluate arithmetic expressions and, furthermore, 
where we can store the results of our computations in variables.  

In [ ]:
!cat -n Calculator.g4

First, we have to generate both the scanner and the parser.  

In [ ]:
!antlr4 -Dlanguage=Python3 Calculator.g4

We can use the system command `ls` to see which files have been generated by <span style="font-variant:small-caps;">Antlr</span>.
If you are using a windows system you have to use the command `dir` instead.

In [ ]:
!ls -l

The files `CalculatorLexer.py` and `CalculatorParser.py` contain the generated scanner and parser, respectively.  We have to import these files.  Furthermore, the runtime of 
<span style="font-variant:small-caps;">Antlr</span>
needs to be imported.

In [ ]:
from CalculatorLexer  import CalculatorLexer
from CalculatorParser import CalculatorParser
import antlr4

Let us parse and evaluate the input that we read from a prompt.

In [ ]:
def main():
    parser        = CalculatorParser(None) # generate parser without lexer
    parser.Values = {}
    line          = input('> ')
    while line != '':
        input_stream  = antlr4.InputStream(line)
        lexer         = CalculatorLexer(input_stream)
        token_stream  = antlr4.CommonTokenStream(lexer)
        parser.setInputStream(token_stream)
        parser.start()
        line = input('> ')
    return parser.Values

In [ ]:
main()

In [ ]:
!rm *.py *.tokens *.interp
!rm -r __pycache__/

In [ ]:
!ls -l